In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

### Create a dataset for 2016 with 1200 rows, 100 companies, and a row for each. 

## Build Information About Each Company:

#### Each business has a unique ID, a set location, and a set number of employees. Also we are going to add a value to determine the order of magnitude of the revenue the company will be making. Rarely are the revenue amounts different by dignificant orders of magnitude from month to month in an established business. We will first generate that information and store it for use later:

In [2]:
# Start with some stable information abotu each company
def generate_company_info(company_id):
    location = random.choices(['A', 'B', 'C', 'D', 'E'], weights=[.25, .20, .10, .05, .04])[0]
    number_of_employees = random.randint(10,1000)
    rev_mag = random.choices([1,2,3,4,5])[0]
    return {'CompanyID':company_id,
            'Region':location, 
            'Employees':number_of_employees, 
            'Rev_Mag':rev_mag}

In [3]:
base_company_info = [generate_company_info(i) for i in range(0,100)]
base_company_info_df = pd.DataFrame(base_company_info)
base_company_info_df.index = base_company_info_df.CompanyID
base_company_info_df.drop(columns='CompanyID', inplace=True)

In [4]:
base_company_info_df.head()

,Employees,Region,Rev_Mag
CompanyID,,,
0,347,A,4
1,427,B,5
2,817,A,3
3,771,B,3
4,445,C,3


In [5]:
#Establish the date range we will be indexing with
dates = pd.date_range('1/1/2016', periods=12, freq='M') 

In [6]:
dates

DatetimeIndex(['2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
               '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31',
               '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31'],
              dtype='datetime64[ns]', freq='M')

In [7]:
#create a multi-dimensional index for future great ideas.
index = pd.MultiIndex.from_product([dates, base_company_info_df.index], names=['Date', 'CompanyID'])

In [8]:
#initialize the dataframe
p_and_l_df = pd.DataFrame([], index=index)

In [9]:
# Merge the static data into the empty dataframe we just created
p_and_l_df = pd.merge(p_and_l_df, base_company_info_df, how='left', left_index=True, right_index=True)

In [10]:
# Have a peak...
p_and_l_df.head(10)

Employees Region  Rev_Mag
Date       CompanyID                           
2016-01-31 0                347      A        4
           1                427      B        5
           2                817      A        3
           3                771      B        3
           4                445      C        3
           5                977      A        5
           6                482      A        1
           7                108      B        1
           8                476      D        4
           9                511      E        5

In [11]:
p_and_l_df.iloc[0]

Employees    347
Region         A
Rev_Mag        4
Name: (2016-01-31 00:00:00, 0), dtype: object

### Generate monthly revenue, expenses, and profit.

### For the sake of argument, let's assume that these companies have expenses and revenue that are at least in the same order of magnitude most of the time to prevent outrageous combinations of revenue and expenses.

In [12]:
def generate_revenue(magnitude):
    min_rev = 10 ** magnitude
    max_rev = (10 ** (magnitude + 1)) - 0.01
    return(np.round(random.uniform(min_rev, max_rev),2))

def generate_expenses(magnitude):
    #Sometimes expenses are significantly lower or greater than revenues.
    new_mag = random.choices([magnitude, magnitude - 1, magnitude + 1], weights=[0.9, 0.05, 0.05])[0]
    if new_mag >= 5: #stay within dataset sonstraints
        max_expenses = 500000.00
    else:
        max_expenses = (10 ** (new_mag + 1)) - 0.01
    min_expenses = 10 ** new_mag
    return(np.round(random.uniform(min_expenses, max_expenses),2))

In [13]:
# just a little test of the functions
print(generate_revenue(4),generate_expenses(4))

74604.47 445340.18


In [14]:
p_and_l_df['Revenue'] = p_and_l_df.Rev_Mag.apply(generate_revenue)
p_and_l_df.Revenue.apply(np.round,2)
p_and_l_df['Expenses'] = p_and_l_df.Rev_Mag.apply(generate_expenses)
p_and_l_df.Expenses.apply(np.round,2)
p_and_l_df['Profit'] = p_and_l_df['Revenue'] - p_and_l_df['Expenses']
p_and_l_df.Profit.apply(np.round,2)
p_and_l_df['LossFlag'] = p_and_l_df['Profit'].apply(lambda x: 1 if x < 0 else 0)

In [15]:
p_and_l_df.head()

Employees Region  Rev_Mag    Revenue   Expenses  \
Date       CompanyID                                                    
2016-01-31 0                347      A        4   59787.83   35665.13   
           1                427      B        5  839843.11  186981.75   
           2                817      A        3    2947.15    8697.11   
           3                771      B        3    9831.70    3352.14   
           4                445      C        3    5212.89    7348.36   

                         Profit  LossFlag  
Date       CompanyID                       
2016-01-31 0           24122.70         0  
           1          652861.36         0  
           2           -5749.96         1  
           3            6479.56         0  
           4           -2135.47         1

### Define the company valuation:

In [16]:
p_and_l_df['YTD_Profit'] = np.nan

In [17]:
# This will take a bit to run!
for i in range(0,100):
    #Calculate the year to date profit for the company
    p_and_l_df.loc[(dates,i),'YTD_Profit'] = p_and_l_df.loc[(dates,i),'Profit'].cumsum()
    # Determine if a cumulitive 3% of YTD Profit is greater than 10% of YTD Profit, and adjust valuation accordingly
    p_and_l_df.loc[(dates,i),'3_%_of_YTD_Profit'] = p_and_l_df.loc[(dates,i),'Profit'].cumsum() * 0.03
    p_and_l_df.loc[(dates,i),'10_%_of_YTD_Profit'] = p_and_l_df.loc[(dates,i),'Profit'].cumsum() * 0.10
    p_and_l_df.loc[(dates,i),'Sum_of_3_%_of_YTD_Profit'] = p_and_l_df.loc[(dates,i),'3_%_of_YTD_Profit'].cumsum()
    p_and_l_df.loc[(dates,i),'Sum_of_3_%_>10_%_of_YTD_Profit'] = p_and_l_df.loc[(dates,i),'Sum_of_3_%_of_YTD_Profit'] > p_and_l_df.loc[(dates,i),'10_%_of_YTD_Profit']
    p_and_l_df.loc[(dates,i),'BusinessValuation'] = p_and_l_df.apply((lambda x: x['Sum_of_3_%_of_YTD_Profit'] if x['Sum_of_3_%_>10_%_of_YTD_Profit'] else x['10_%_of_YTD_Profit']), axis=1)


In [18]:
# Test what we've done.
p_and_l_df.BusinessValuation = p_and_l_df.BusinessValuation.apply(np.round,2)
p_and_l_df['BusinessValuation'].head(10)

Date        CompanyID
2016-01-31  0             2412.0
            1            65286.0
            2             -172.0
            3              648.0
            4              -64.0
            5            52622.0
            6                2.0
            7                1.0
            8             3078.0
            9            16781.0
Name: BusinessValuation, dtype: float64

### Determine the Closed Flag for these companies:

As the year progresses, a company has a 0.5% chance of going out of business if two of the months have had net losses, and that percentage jumps to 10% if three of the momnths have had net losses.

In [19]:
# Since we're only talking about 1% of the total records for each item, that's simply 1 value each month from a random entry.
# p_and_l_df['ClosedFlag'] = np.nan
for i in range(0,100):
    #Calculate the year to date sum of the loss flag for each 
    p_and_l_df.loc[(dates,i),'LossFlagSum'] = p_and_l_df.loc[(dates,i),'LossFlag'].cumsum()
    # Roll the dice on the closing status.
    p_and_l_df.loc[(dates,i),'ClosedFlag'] = p_and_l_df.apply((lambda x: 
                                                               random.choices([0,1], weights=[0.995, 0.005])[0] 
                                                               if x['LossFlagSum'] <= 2 
                                                               else random.choices([0,1], weights=[0.90, 0.10])[0]) , axis=1)



In [20]:
# Also, let's make sure we mark the ClosedFlag = 1 if the business has been closed already!
for i in range(0,100):
    next_val = None
    for d in dates:
        if next_val:
           p_and_l_df.loc[(d,i),'ClosedFlag'] = next_val 
        if p_and_l_df.loc[(d,i),'ClosedFlag'] == 1:
            next_val = 1
            # A company that has been closed no longer has a revenue, expenses, profit, or losses.
            p_and_l_df.loc[(d,i),'Revenue'] = np.nan
            p_and_l_df.loc[(d,i),'Expenses'] = np.nan
            p_and_l_df.loc[(d,i),'Profit'] = np.nan
            p_and_l_df.loc[(d,i),'LossFlag'] = np.nan
            p_and_l_df.loc[(d,i),'BusinessValuation'] = np.nan
        else:
            next_val = 0

## Add in missing values.


In [21]:
for d in dates:
    p_and_l_df.loc[(d,np.random.randint(0,100)),'Revenue'] = np.nan
    p_and_l_df.loc[(d,np.random.randint(0,100)),'Expenses'] = np.nan
    p_and_l_df.loc[(d,np.random.randint(0,100)),'Profit'] = np.nan
    p_and_l_df.loc[(d,np.random.randint(0,100)),'LossFlag'] = np.nan

In [22]:
# How many null values do we have now?
p_and_l_df[p_and_l_df.columns[p_and_l_df.isnull().any()].tolist()].isnull().sum()


Revenue              199
Expenses             200
Profit               201
LossFlag             199
BusinessValuation    189
dtype: int64

## Finally, Write the dataset file

In [23]:
p_and_l_df[['Revenue', 'Expenses', 'Profit', 'LossFlag', 'Employees', 'Region', 'BusinessValuation', 'ClosedFlag']].to_csv('Profit_and_Loss_Challenge_Dataset.csv')